# Jupyter Notebook Testing

Jupyter Notebooks use cells that can easily be re ran and is very convenient for testing things out.
It's important to note that to be able to run these cells correctly, the dependencies need to be properly installed.
This means that the interpreter being used must point to the Python the one that is in your [venv](./.venv) created by poetry for this project.

**Note**: if you make changes to the source code, you will need to press `Restart` above to restart the kernel to ensure the changes occur here.
![image](../pictures/restartJupyterKernel.png)

In [4]:
import logging

logging.basicConfig(level=logging.INFO)
from engine.types.industry_type import IndustryType
from engine.types.demographic import Demographic


policies = {
    "corporate_income_tax": {itype.value: 0.0 for itype in IndustryType},
    "personal_income_tax": 0.0,
    "sales_tax": {itype.value: 0.0 for itype in IndustryType},
    "property_tax": 0.0,
    "tariffs": {itype.value: 0.0 for itype in IndustryType},
    "subsidies": {itype.value: 0.0 for itype in IndustryType},
    "minimum_wage": 0.0,
}

demographics = {
    demo.value: {
        "income": {"mean": 300 + (i * 500), "sd": 100},
        "proportion": 1 / len(Demographic),
        "unemployment_rate": 0.1 - (i * 0.02),
        "spending_behavior": {
            itype.value: 1 / len(IndustryType) for itype in IndustryType
        },
        "current_money": {"mean": 500 + (i * 2000), "sd": 100},
    }
    for i, demo in enumerate(Demographic)
}

In [5]:
from engine.interface.controller import ModelController
controller = ModelController()
model_id = controller.create_model(num_people=100,demographics=demographics, starting_policies=policies,inflation_rate=0.001)

In [6]:
controller.step_model(model_id,time=10)

INFO:MESA.mesa.model:calling model.step for timestep 1 
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.

In [7]:
indicators = controller.get_indicators(model_id, start_time=1, end_time=0)
print(indicators)

    Week  Unemployment  GDP  IncomePerCapita  MedianIncome  HooverIndex  \
1      1           1.0    0        51.619269     49.658328            0   
2      2           1.0    0        52.619269     50.658328            0   
3      3           1.0    0        53.619269     51.658328            0   
4      4           1.0    0        54.619269     52.658328            0   
5      5           1.0    0        55.619269     53.658328            0   
6      6           1.0    0        56.619269     54.658328            0   
7      7           1.0    0        57.619269     55.658328            0   
8      8           1.0    0        58.619269     56.658328            0   
9      9           1.0    0        59.619269     57.658328            0   
10    10           1.0    0        60.619269     58.658328            0   

    LorenzCurve  
1             0  
2             0  
3             0  
4             0  
5             0  
6             0  
7             0  
8             0  
9           